In [1]:
STRAVA_DOMAIN = "https://www.strava.com"
API_VERSION = "/api/v3"

UID = 15972102
ACTIVITY_ENDPOINT = f"{API_VERSION}/activities/{UID}?include_all_efforts=false"

access_token = "93db078e76f283458f58513e16dc200fbdbba83d"
refresh_token = "05867993a2d0c5b60c51653636a9c295348551f3"

headers = {"Authorization": "Bearer {}".format(access_token)}



In [2]:
import aiohttp
import asyncio

MAX_PAGE = 2
PER_PAGE = 200
PAGE_CONCURRENCY = 10

PAGE_SIZE = 10
ACTIVITY_LIST_ENDPOINT = f"{API_VERSION}/athlete/activities"


params = { "per_page": PER_PAGE }
empty = []

last_page = 1
async def get_page(session, p):
    if p > last_page:
        print(f"  page {p} aborted")
        return p, empty
    
    async with session.get(ACTIVITY_LIST_ENDPOINT, params={**params, "page": p}) as r:
        body = await r.json()
        return p, body
    
done = False
summaries = []
async with aiohttp.ClientSession(STRAVA_DOMAIN, headers=headers) as session:
    while not done and (last_page < MAX_PAGE):
        p0 = last_page
        p1 = last_page + PAGE_CONCURRENCY
        last_page = p1
        print(f"requesting pages {p0} - {p1}")
        requests = [get_page(session, p) for p in range(p0, p1)]
        for coro in asyncio.as_completed(requests):
            p, body = await coro
            m = len(body)
            print(f"page {p}: {m}")
            if m:
                summaries.extend(body)
                
            if m < PER_PAGE:
                last_page = min(p, last_page)
                done = True
                print(f" last page: {last_page}")
        
    


requesting pages 1 - 11
page 1: 200
page 2: 200
page 8: 200
page 7: 200
page 6: 200
page 3: 200
page 4: 200
page 5: 200
page 9: 200
page 10: 200


In [3]:
A = summaries[0]

In [18]:
import numpy as np

def rlld_encode(vals, neg_vals=False):
    my_dtype = np.int8 if neg_vals else np.uint8
    marker = -127 if neg_vals else 0xff 
    diffs = np.diff(np.array(vals))

    ## TODO: pick up here
    encoded = []
    tup = [marker, 0, 0]
    for a, b in zip(diffs, diffs[1:]):
        if a == b:
            tup[1] += 1
            else:
                pair = [a, 2]
        else:
            if pair:
                if pair[1] > 2:
                    encoded.append(pair)
                else:
                    encoded.extend(2 * [pair[0]])
                pair = None
            else:
                encoded.append(a)
    if pair:
        encoded.append(pair)
    else:
        encoded.append(b)
 
    return np.array(encoded, my_dtype)
        
        
#     def stream_decode(rll_encoded, first_value=0):
#         running_sum = first_value
#         out_list = [first_value]

#         for el in rll_encoded:
#             if isinstance(el, list) and len(el) == 2:
#                 val, num_repeats = el
#                 for i in range(num_repeats):
#                     running_sum += val
#                     out_list.append(running_sum)
#             else:
#                 running_sum += el
#                 out_list.append(running_sum)

#         return out_list


In [19]:
import numpy as np
import aiohttp
import asyncio
import time
import polyline

activity_ids = [6611297682, 794592655]

ACTIVITY_STREAM_PARAMS = {
    "keys": "latlng,altitude,time",
    "key_by_type": "true",
    "series_type": "time",
    "resolution": "high"
}

def streams_endpoint(activity_id):
    return f"{API_VERSION}/activities/{activity_id}/streams"

def StravaSession():
    return aiohttp.ClientSession(STRAVA_DOMAIN, headers=headers)

async def get_streams(session, activity_id):
    t0 = time.perf_counter()
    async with session.get(streams_endpoint(activity_id), params=ACTIVITY_STREAM_PARAMS) as r:
        streams = await r.json()
        result = {
            "dt": rlld_encode(streams["time"]["data"]),
            "da": rlld_encode(streams["altitude"]["data"], neg_vals=True),
            "pt": polyline.encode(streams["latlng"]["data"])
        } if streams else None
    
    return activity_id, result, time.perf_counter() - t0

t0 = time.perf_counter()
result = {}
async with StravaSession() as session:
    requests = [get_streams(session, aid) for aid in activity_ids]
    for coro in asyncio.as_completed(requests):
        aid, nparr, dt = await coro
        if nparr is not None:
            result[str(aid)] = nparr
            print(f"{aid} took {dt:.2f}")
print(f"elapsed: {time.perf_counter() - t0}")

6611297682 took 0.46
794592655 took 0.46
elapsed: 0.46823160299936717


In [20]:
result

{'6611297682': {'dt': array([  1, 255, 255,   1, 255, 231,   1], dtype=uint8),
  'da': array([ 0, -1, 20,  0,  1,  0, -1, 17,  0,  0,  0, -1, 12,  0,  1,  0, -1,
         55,  0, -1,  0, -1, 11,  0,  0,  0,  0,  0,  0,  0, -1, 24,  0,  1,
          0, -1, 14,  0,  0,  0, -1, 10,  0,  0,  0, -1,  4,  0, -1,  0,  0,
          0,  1,  0, -1,  8,  0, -1,  0, -1,  4,  0,  1,  0, -1, 12,  0,  0,
          0, -1,  8,  0,  0,  0,  0,  0, -1,  0,  0,  0,  1,  0, -1,  4,  0,
         -1,  0, -1, 14,  0,  0,  0, -1, 11,  0,  0,  0, -1,  9,  0,  1,  0,
         -1, 14,  0,  0,  0, -1, 10,  0,  1,  0, -1,  4,  0, -1,  0, -1, 18,
          0,  0,  0, -1, 16,  0,  0,  0, -1, 10,  0,  0,  0,  0,  0,  0,  0,
         -1, 24,  0,  0,  0, -1,  4,  0, -1,  0,  0,  0,  0,  0, -1,  6,  0,
          0,  0, -1,  8,  0,  0,  0,  0,  0,  1,  0, -1,  6,  0,  0,  0, -1,
         20,  0,  0,  0, -1, 16,  0, -1,  0,  0,  0,  1,  0], dtype=int8),
  'pt': 'yxzeFjpzhV@@?AA?BD@@??@BB@B@?@BB?B@D?D?B?@?BADAB?@?D?D?D@B@D@

In [7]:
import msgpack_numpy as mnp
import msgpack
result_enc = msgpack.packb(result, default=mnp.encode)

In [8]:
result_enc

b"\x82\xa9794592655\x83\xa2dt\x85\xc4\x02nd\xc3\xc4\x04type\xa3|u1\xc4\x04kind\xc4\x00\xc4\x05shape\x91\xcd\x01\xf6\xc4\x04data\xc5\x01\xf6\t-\x0c\x07\x07\x07\x07\x07\x07\x07\x07\x07\x07\x07\x08\x07\x07\x07\x08\x08\x08\x08\x08\x08\x08\x08\x07\x07\x07\x08\x07\x07\x07\x07\x07\x08\x08\x08\x08\x06\x07\x07\x07\x07\x07\x07\x07\x06\x07\x08\t\t\x07\x07\x07\x07\x07\x07\x07\x07\x07\x07\x07\x07\x07\x07\x07\x07\x07\x07\x08\x07\x07\x07\x07\x08\x07\x06\x07\x06\x07\x07\x08\x07\x07\x07\x05\x07\x06\x06\x07\x08\n\t\t\x08\x07\x07\x08\x07\x08\x08\x08\t\t\x07\x08\n\x08\x08\x08\x07\x06\xff\x04\t\x0b\n\x08\x07\x06\x07\x07\x07\xff\x04\t\x08\x07\x08\n\x07\x07\x06\x06\x07\x07\x05\x06\x08\x08\x08\x07\x06\x06\x06\x07\x06\x06\x06\x07\x06\x06\x08\x05\x06\x06\x08\x07\x07\x0c\x12\r\x0e\x08'\x04\x13\x05\x13\x04\x0e\x0c\x04\x11\x03\x05\x10\xff\x04\t\x02\x04\r\r\x0b\x02\n\x06\x08\t\t\x04\x02\x1e\x08\t\t\x0c\x03\x08\t\t\x06\x07\x1a\x14\n\x06\x18\x0b\n\n\x07\x16\x10\x10\x13\x07\x05\x02\n\x07\x06\x08\x03\x0b\x0b\x0c\t\t\x0

In [9]:
msgpack.unpackb(result_enc)

{'794592655': {'dt': {b'nd': True,
   b'type': '|u1',
   b'kind': b'',
   b'shape': [502],
   b'data': b"\t-\x0c\x07\x07\x07\x07\x07\x07\x07\x07\x07\x07\x07\x08\x07\x07\x07\x08\x08\x08\x08\x08\x08\x08\x08\x07\x07\x07\x08\x07\x07\x07\x07\x07\x08\x08\x08\x08\x06\x07\x07\x07\x07\x07\x07\x07\x06\x07\x08\t\t\x07\x07\x07\x07\x07\x07\x07\x07\x07\x07\x07\x07\x07\x07\x07\x07\x07\x07\x08\x07\x07\x07\x07\x08\x07\x06\x07\x06\x07\x07\x08\x07\x07\x07\x05\x07\x06\x06\x07\x08\n\t\t\x08\x07\x07\x08\x07\x08\x08\x08\t\t\x07\x08\n\x08\x08\x08\x07\x06\xff\x04\t\x0b\n\x08\x07\x06\x07\x07\x07\xff\x04\t\x08\x07\x08\n\x07\x07\x06\x06\x07\x07\x05\x06\x08\x08\x08\x07\x06\x06\x06\x07\x06\x06\x06\x07\x06\x06\x08\x05\x06\x06\x08\x07\x07\x0c\x12\r\x0e\x08'\x04\x13\x05\x13\x04\x0e\x0c\x04\x11\x03\x05\x10\xff\x04\t\x02\x04\r\r\x0b\x02\n\x06\x08\t\t\x04\x02\x1e\x08\t\t\x0c\x03\x08\t\t\x06\x07\x1a\x14\n\x06\x18\x0b\n\n\x07\x16\x10\x10\x13\x07\x05\x02\n\x07\x06\x08\x03\x0b\x0b\x0c\t\t\x06\x0c\x07\x10\x03\x07\n\t\t\x08\x0

In [10]:
result

{'794592655': {'dt': array([  9,  45,  12,   7,   7,   7,   7,   7,   7,   7,   7,   7,   7,
           7,   8,   7,   7,   7,   8,   8,   8,   8,   8,   8,   8,   8,
           7,   7,   7,   8,   7,   7,   7,   7,   7,   8,   8,   8,   8,
           6,   7,   7,   7,   7,   7,   7,   7,   6,   7,   8,   9,   9,
           7,   7,   7,   7,   7,   7,   7,   7,   7,   7,   7,   7,   7,
           7,   7,   7,   7,   7,   8,   7,   7,   7,   7,   8,   7,   6,
           7,   6,   7,   7,   8,   7,   7,   7,   5,   7,   6,   6,   7,
           8,  10,   9,   9,   8,   7,   7,   8,   7,   8,   8,   8,   9,
           9,   7,   8,  10,   8,   8,   8,   7,   6, 255,   4,   9,  11,
          10,   8,   7,   6,   7,   7,   7, 255,   4,   9,   8,   7,   8,
          10,   7,   7,   6,   6,   7,   7,   5,   6,   8,   8,   8,   7,
           6,   6,   6,   7,   6,   6,   6,   7,   6,   6,   8,   5,   6,
           6,   8,   7,   7,  12,  18,  13,  14,   8,  39,   4,  19,   5,
          19,   4, 